# 데이터 정보

In [19]:
import numpy as np
import cv2
import torch
import pandas as pd
import re
import pickle as pkl
import json
import glob
import shutil
import os
import json
import pathlib as pl
from typing import List
from tqdm import trange, tqdm
from glob import glob
from collections import Counter
from sklearn.model_selection import train_test_split

In [20]:
# Get all json files in the labeled data
data_path = "/data/NIA50/50-1/data/labeled_test"
data_path: pl.Path = pl.Path(data_path)

# Get all json files path in the labeled data
label_path: List[str] = sorted([str(p) for p in data_path.rglob("**/*.json")])

# Get all txt files in the labeled data
txtfile_path: List[str] = sorted([p for p in data_path.rglob("**/*.txt")])

In [21]:
# # save label_path and txtfile_path to pickle file
# with open("label_path.pkl", "wb") as f:
#     pkl.dump(label_path, f)
# with open("txtfile_path.pkl", "wb") as f:
#     pkl.dump(txtfile_path, f)


In [22]:
label_path = pkl.load(open("label_path.pkl", "rb"))

import shutil

# create an empty folder named label move all json files to label folder
for p in label_path:
    shutil.move(p, p.replace("labeled_test", "label"))